# Homework w03d03

Use again the auto dataset. This time we want to use cross validation to assess the quality of various models.
1. Split the dataset into a training set and a test set. 
1. Using linear regression, calculate the mean squared error (MSE) and the R-squared for all possible single predictor models on the training set and the test set. Which one performs best?
1. Using linear regression, calculate the mean squared error (MSE) and the R-squared for fitting mpg on an polynomial in horsepower with degree varying from 1 to 5 on the trainig set and the test set. Which one performs best? 
1. Perform an ordinary least squares fit to mpg with all features except name, calculate the mean squared error (MSE) and the R-squared on the trainig set and the test set.

In [1]:
# load libraries
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# read in data indicating the correct path
df = pd.read_csv('../day1/Auto.csv')
# drop missing values and convert datatype
df.dropna(inplace=True)
df.drop(list(df.index[df.horsepower=='?']),inplace=True)
df.horsepower = df.horsepower.astype(int)

Define your response y and feature matrix X and 
use the train_test_split function to split the data set into a training and a test set.
Use 20 percent of the original dataset as the test set.

### 1. Split the dataset into a training set and a test set. 

In [2]:
X = df.ix[:,1:8]
y = df.mpg
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=10)
df_train = X_train.join(y_train)
df_test = X_test.join(y_test)

### 2. Using linear regression, calculate the mean squared error (MSE) and the R-squared for all possible single predictor models on the training set and the test set. Which one performs best?

In [3]:
import statsmodels.formula.api as smf

# Run on the training set
max_train = -1
min_train = 10**10
form_max_train = ""
max_test = -1
min_test = 10**10
form_max_test = ""
for v1 in X_train:
    formula = "mpg ~ " + v1
    model=smf.ols(formula, df_train).fit()
    predictions=model.predict(X_train)
    rsq_train = model.rsquared
    print("%s Train r^2: %f" % (formula, rsq_train))
    print("\t\tTrain MSE: %f" % (mean_squared_error(y_train,predictions)))
    predictions_test = model.predict(X_test)
    rsq_test = r2_score(y_test,predictions_test)
    print("\t\tTest r^2: %f" % (rsq_test))
    print("\t\tTest MSE: %f" % mean_squared_error(y_test,predictions_test))
    if rsq_train > max_train:
        max_train = rsq_train
        form_max_train = formula
    if mean_squared_error(y_train,predictions) < min_train:
        min_train =  mean_squared_error(y_train,predictions)
        form_min_train = formula
    if rsq_test > max_test:
        max_test = rsq_test
        form_max_test = formula
    if mean_squared_error(y_test,predictions_test) < min_test:
        min_test = mean_squared_error(y_test,predictions_test)
        form_min_test = formula

print("\nBest formula maximizing r^2 on training set: %s : %f" % (form_max_train,max_train))
print("\nBest formula minimizing MSE on training set: %s : %f" % (form_min_train,min_train))
print("\nBest formula maximizing r^2 on testing set: %s : %f" % (form_max_test,max_test))
print("\nBest formula minimizing MSE on testing set: %s : %f" % (form_min_test,min_test))

mpg ~ cylinders Train r^2: 0.605528
		Train MSE: 23.696967
		Test r^2: 0.586027
		Test MSE: 25.399000
mpg ~ displacement Train r^2: 0.640203
		Train MSE: 21.613955
		Test r^2: 0.666215
		Test MSE: 20.479160
mpg ~ horsepower Train r^2: 0.605326
		Train MSE: 23.709129
		Test r^2: 0.590135
		Test MSE: 25.146933
mpg ~ weight Train r^2: 0.688836
		Train MSE: 18.692441
		Test r^2: 0.695988
		Test MSE: 18.652400
mpg ~ acceleration Train r^2: 0.157431
		Train MSE: 50.615355
		Test r^2: 0.220318
		Test MSE: 47.836817
mpg ~ year Train r^2: 0.319669
		Train MSE: 40.869337
		Test r^2: 0.372025
		Test MSE: 38.528921
mpg ~ origin Train r^2: 0.332835
		Train MSE: 40.078413
		Test r^2: 0.242698
		Test MSE: 46.463719

Best formula maximizing r^2 on training set: mpg ~ weight : 0.688836

Best formula minimizing MSE on training set: mpg ~ weight : 18.692441

Best formula maximizing r^2 on testing set: mpg ~ weight : 0.695988

Best formula minimizing MSE on testing set: mpg ~ weight : 18.652400


In both cases we identify weight as the variable which best predicts mpg.

### 3. Using linear regression, calculate the mean squared error (MSE) and the R-squared for fitting mpg on an polynomial in horsepower with degree varying from 1 to 5 on the trainig set and the test set. Which one performs best? 

In [4]:
def polynomials(X, degree=1):
    # See also numpy.vander
    array = [X]
    y = X
    for i in range(2, int(degree + 1)):
        y = y * X
        array.append(y)
    return np.array(array).transpose()



In [5]:
X_train_hp = X_train["horsepower"]
X_test_hp = X_test["horsepower"]

In [6]:
import statsmodels.api as sm

# Run on the training set
max_train = -1
min_train = 10**10
best_pol_rs2_train=0
best_pol_mse_train=0
max_test = -1
min_test = 10**10
best_pol_rs2_test=0
best_pol_mse_test=0


for i in range(1,6):
    X_train_hp_pol = polynomials(X_train_hp,i)
    X_train_hp_pol = sm.add_constant(X_train_hp_pol)
    X_test_hp_pol = polynomials(X_test_hp,i)
    X_test_hp_pol = sm.add_constant(X_test_hp_pol)
    model = sm.OLS(y_train, X_train_hp_pol).fit() # OLS stands for ordinary least squares
    predictions=model.predict(X_train_hp_pol)
    rsq_train = r2_score(y_train,predictions)
    print("Polynomial %d \t Train r^2: %f" % (i,rsq_train))
    print("\t\tTrain MSE: %f" % (mean_squared_error(y_train,predictions)))
    predictions_test=model.predict(X_test_hp_pol)
    rsq_test = r2_score(y_test,predictions_test)
    print("\t\tTest r^2: %f" % (rsq_test))
    print("\t\tTest MSE: %f" % (mean_squared_error(y_test,predictions_test)))
    if rsq_train > max_train:
        max_train = rsq_train
        best_pol_rs2_train=i
    if mean_squared_error(y_train,predictions) < min_train:
        min_train = mean_squared_error(y_train,predictions)
        best_pol_mse_train = i
    if rsq_test > max_test:
        max_test = rsq_test
        best_pol_rs2_test=i
    if mean_squared_error(y_test,predictions_test) < min_test:
        min_test = mean_squared_error(y_test,predictions_test)
        best_pol_mse_test = i

print("\nBest degree polynomial is %d with r^2 on training set: %f" % (best_pol_rs2_train,max_train))
print("\nBest degree polynomial is %d with MSE on training set: %f" % (best_pol_mse_train,min_train))
print("\nBest degree polynomial is %d with r^2 on testing set: %f" % (best_pol_rs2_test,max_test))
print("\nBest degree polynomial is %d with MSE on testing set: %f" % (best_pol_mse_test,min_test))

Polynomial 1 	 Train r^2: 0.605326
		Train MSE: 23.709129
		Test r^2: 0.590135
		Test MSE: 25.146933
Polynomial 2 	 Train r^2: 0.681084
		Train MSE: 19.158114
		Test r^2: 0.696033
		Test MSE: 18.649665
Polynomial 3 	 Train r^2: 0.681108
		Train MSE: 19.156716
		Test r^2: 0.697054
		Test MSE: 18.587045
Polynomial 4 	 Train r^2: 0.682991
		Train MSE: 19.043592
		Test r^2: 0.693747
		Test MSE: 18.789948
Polynomial 5 	 Train r^2: 0.690188
		Train MSE: 18.611224
		Test r^2: 0.700883
		Test MSE: 18.352103

Best degree polynomial is 5 with r^2 on training set: 0.690188

Best degree polynomial is 5 with MSE on training set: 18.611224

Best degree polynomial is 5 with r^2 on testing set: 0.700883

Best degree polynomial is 5 with MSE on testing set: 18.352103


As expected in the train case the higher order polynomial performs better for both MSE and r2, but most likely it is overfitting the model.

This should show when using this same model on the test data were we expect high polynomial to have poor performance. However here is not the case, it appears high order polynomial is showing good results with the test data. This may show an error in the model construction.

### 4. Perform an ordinary least squares fit to mpg with all features except name, calculate the mean squared error (MSE) and the R-squared on the trainig set and the test set.

In [8]:
formula = "mpg ~ cylinders + displacement + horsepower + weight + acceleration + year + origin"
model=smf.ols(formula, df_train).fit()
predictions=model.predict(X_train)
rsq_train = model.rsquared
print("%s Train r^2: %f" % (formula, rsq_train))
print("\t\tTrain MSE: %f" % (mean_squared_error(y_train,predictions)))
predictions_test = model.predict(X_test)
rsq_test = r2_score(y_test,predictions_test)
print("\t\tTest r^2: %f" % (rsq_test))
print("\t\tTest MSE: %f" % mean_squared_error(y_test,predictions_test))

mpg ~ cylinders + displacement + horsepower + weight + acceleration + year + origin Train r^2: 0.828055
		Train MSE: 10.329184
		Test r^2: 0.784712
		Test MSE: 13.208830


Here the outcome is expected. We have better r2 and MSE on the training than on the test set.